In [ ]:
from __future__ import annotations
import os
import sys
import json
from importlib import reload
from typing_extensions import TypedDict, NotRequired
from aws_lambda_powertools import Logger
from aws_lambda_powertools.event_handler import Response, content_types
from requests import status_codes
from code.handler_lambda.src.helpers.network import APIS, make_request, RequestResponse
from code.handler_lambda.src.helpers.datetime import jenkins_build_datetime

BITBUCKET_WORKSPACE = os.getenv("BITBUCKET_WORKSPACE", "workspace")
BITBUCKET_REPO_SLUG = os.getenv("BITBUCKET_REPO_SLUG", "repo")

JENKINS_AT_JOB_NAME = os.getenv("JENKINS_AT_JOB_NAME", 'job')
JENKINS_PR_JOB_NAME = os.getenv("JENKINS_PR_JOB_NAME", 'job')

logger = Logger()

In [ ]:
for module_key in [key for key in sys.modules.keys() if "code.handler_lambda" in key]:
    reload(sys.modules[module_key])

In [ ]:
from code.handler_lambda.src.helpers.network import APIS, make_request, RequestResponse
from code.handler_lambda.src.helpers.datetime import jenkins_build_datetime

In [ ]:
class SingularChangeLeadTimeData(TypedDict):
    # numberOfDeployments: str
    # latestBuildDatetime: str
    # firstBuildDatetime: str
    # daysBetweenLatestAndFirstBuild: int
    no: str

class SingularChangeLeadTimeResult(TypedDict):
    success: bool
    message: NotRequired[str | None]
    data: NotRequired[SingularChangeLeadTimeData | None]

In [ ]:
def create_error_singular_change_lead_time_result(message: str) -> SingularChangeLeadTimeResult:
    return SingularChangeLeadTimeResult(
        {
            "success": False,
            "message": message
        }
    )

def build_failed_network_request_error_message(api: APIS):
    api_enum = APIS(api)
    if ("JENKINS" in api_enum.name):
        return "jenkins"
    elif ("BITBUCKET" in api_enum.name):
        return "bitbucket"
    else:
        return ""

def handle_failed_network_request(api: APIS, url: str, response: RequestResponse):
    logger.error(
        build_failed_network_request_error_message(api),
        url=url,
        response=response
    )
    return SingularChangeLeadTimeResult(
        {
            "success": False,
            "message": response["message"]
            if "message" in response
            and response["message"]
            else "Error: a network request failed",
        }
    )


In [ ]:
def extract_status_of_parent_commit_url(pull_request):
    parent_commit_hash = pull_request["merge_commit"]["parents"][0]["hash"]
    parent_commit_hash_url = pull_request["merge_commit"]["parents"][0]["links"][
        "html"
    ]["href"]
    statuses_of_parent_commit_url = pull_request["merge_commit"]["parents"][0][
        "links"
    ]["statuses"]["href"]

    return parent_commit_hash, parent_commit_hash_url, statuses_of_parent_commit_url

def fetch_last_build_of_parent_commit_display_url(statuses_of_parent_commit_url: str) -> RequestResponse:
    statuses_of_parent_commit_specific_fields_url = f"{statuses_of_parent_commit_url}?fields=values.key,values.type,values.state,values.name,values.url"

    logger.debug("making request to get the statuses of the commit before the most recent PR", url=statuses_of_parent_commit_specific_fields_url)

    statuses_of_parents_commit_response = make_request(
        APIS.DIRECT_BITBUCKET, statuses_of_parent_commit_specific_fields_url
    )

    if not statuses_of_parents_commit_response["success"]:
        return handle_failed_network_request(APIS.DIRECT_BITBUCKET, statuses_of_parent_commit_specific_fields_url, statuses_of_parents_commit_response)

    logger.debug(
        "successful request to bitbucket to get the statuses",
        url=statuses_of_parent_commit_specific_fields_url,
        response=statuses_of_parents_commit_response,
    )

    return statuses_of_parents_commit_response

def extract_last_build_of_parent_commit_display_url(statuses_of_parents_commit_response: RequestResponse) -> str:
    return statuses_of_parents_commit_response["data"]["values"][0]["url"]

def fetch_first_jenkins_build_of_current_pull_request_url(last_build_of_parent_commit_display_url: str):
    last_build_of_parent_commit_api_url = last_build_of_parent_commit_display_url.replace(
        "/display/redirect",
        "/api/json?tree=displayName,number,id,fullDisplayName,duration,timestamp,url,inProgress,nextBuild[number,url]",
    )

    logger.debug('making request to get the first build of the commit from the most recent PR', url=last_build_of_parent_commit_api_url)
    last_build_of_parent_commit_response = make_request(
        APIS.DIRECT_JENKINS, last_build_of_parent_commit_api_url
    )

    if not last_build_of_parent_commit_response["success"]:
        return handle_failed_network_request(APIS.DIRECT_JENKINS, last_build_of_parent_commit_api_url, last_build_of_parent_commit_response)

    logger.debug(
        "successful request to jenkins to get the first build of the commit from the most recent PR",
        url=last_build_of_parent_commit_api_url,
        response=last_build_of_parent_commit_response,
    )

    return last_build_of_parent_commit_response

def extract_first_jenkins_build_of_current_pull_request_url(last_build_of_parent_commit_response: RequestResponse):
    return last_build_of_parent_commit_response["data"]["nextBuild"]["url"]

def calculate_lead_time_for_changes(pull_request) -> SingularChangeLeadTimeResult:
    # first section
    # takes in the pull_request and outputs 3 variables: parent_commit_hash, parent_commit_hash_url, statuses_of_parent_commit_url
    try:
        parent_commit_hash, parent_commit_hash_url, statuses_of_parent_commit_url = extract_status_of_parent_commit_url(pull_request)
    except KeyError as err:
        return create_error_singular_change_lead_time_result(f"Key {str(err)} cannot be found in the dict")
    except IndexError as err:
        return create_error_singular_change_lead_time_result(f"Unexpected number of merge commits parents for PR {pull_request.get('id', 'null')} in {BITBUCKET_REPO_SLUG}")
    
    # second section
    # takes in 3 variables: parent_commit_hash, parent_commit_hash_url, statuses_of_parent_commit_url
    # outputs: last_build_of_parent_commit_display_url

    statuses_of_parents_commit_response = fetch_last_build_of_parent_commit_display_url(statuses_of_parent_commit_url)

    try:
        last_build_of_parent_commit_display_url = extract_last_build_of_parent_commit_display_url(statuses_of_parents_commit_response)
    except KeyError as err:
        return create_error_singular_change_lead_time_result(f"Key {str(err)} cannot be found in the dict")
    except IndexError as err:
        return create_error_singular_change_lead_time_result(f"Unexpected number of builds for for commit {parent_commit_hash} in {BITBUCKET_REPO_SLUG}. Visit {parent_commit_hash_url}")

    # third section
    # takes in: last_build_of_parent_commit_display_url
    # outputs first_jenkins_build_of_current_pull_request_url

    last_build_of_parent_commit_response = fetch_first_jenkins_build_of_current_pull_request_url(last_build_of_parent_commit_display_url)

    try:
        first_jenkins_build_of_current_pull_request_url = extract_first_jenkins_build_of_current_pull_request_url(last_build_of_parent_commit_response)
    except KeyError as err:
        return create_error_singular_change_lead_time_result(f"Key {str(err)} cannot be found in the dict")
    except IndexError as err:
        return create_error_singular_change_lead_time_result(f"Unexpected data from {BITBUCKET_REPO_SLUG} staging job. Visit {last_build_of_parent_commit_display_url}")

    # fourth section
    # takes in first_jenkins_build_of_current_pull_request_url
    # outputs first_jenkins_build_of_current_pull_request_id, first_jenkins_build_of_current_pull_request_timestamp
    # 

    first_jenkins_build_of_current_pull_request_apis_url = f"{first_jenkins_build_of_current_pull_request_url}api/json?tree=displayName,number,id,fullDisplayName,duration,timestamp,url,inProgress,nextBuild[number,url]"

    logger.debug('making request to get the the id of the first st build of the commit from the most recent PR', url=first_jenkins_build_of_current_pull_request_apis_url)
    first_jenkins_build_of_current_pull_request = make_request(
        APIS.DIRECT_JENKINS, first_jenkins_build_of_current_pull_request_apis_url
    )

    if not first_jenkins_build_of_current_pull_request["success"]:
        return handle_failed_network_request(APIS.DIRECT_JENKINS, first_jenkins_build_of_current_pull_request_apis_url, first_jenkins_build_of_current_pull_request)

    logger.debug(
        "successful request to jenkins to get the the id of the first st build of the commit from the most recent PR",
        url=first_jenkins_build_of_current_pull_request_apis_url,
        response=first_jenkins_build_of_current_pull_request,
    )

    try:
        first_jenkins_build_of_current_pull_request_id = first_jenkins_build_of_current_pull_request["data"]["id"]
        first_jenkins_build_of_current_pull_request_timestamp = first_jenkins_build_of_current_pull_request["data"]["timestamp"]
    except KeyError as err:
        return create_error_singular_change_lead_time_result(f"Key {str(err)} cannot be found in the dict")
    except IndexError as err:
        return create_error_singular_change_lead_time_result(f"Unexpected data from {first_jenkins_build_of_current_pull_request_apis_url} in {BITBUCKET_REPO_SLUG} staging job. Visit {first_jenkins_build_of_current_pull_request_apis_url}")

    # fifth section

    first_jenkins_build_of_current_pull_request_datetime = jenkins_build_datetime({"timestamp": first_jenkins_build_of_current_pull_request_timestamp})

    # sixth section
    # takes in first_jenkins_build_of_current_pull_request_id

    first_jenkins_at_build_of_current_pull_request_path = f"{JENKINS_AT_JOB_NAME}/api/xml?tree=builds[number,url,actions[causes[upstreamUrl,upstreamBuild]]]&xpath=/workflowJob/build/action/cause[upstreamBuild%20=%20%27{first_jenkins_build_of_current_pull_request_id}%27]/../.."

    logger.debug("making request to get the the id of the first at build of the commit from the most recent PR", path=first_jenkins_at_build_of_current_pull_request_path)
    first_jenkins_at_build_of_current_pull_request = make_request(APIS.JENKINS, first_jenkins_at_build_of_current_pull_request_path)

    if not first_jenkins_at_build_of_current_pull_request["success"]:
        return handle_failed_network_request(APIS.JENKINS, first_jenkins_at_build_of_current_pull_request_path, first_jenkins_at_build_of_current_pull_request)

    logger.debug(
        "successful request to jenkins to get the the id of the first at build of the commit from the most recent PR",
        path=first_jenkins_at_build_of_current_pull_request_path,
        response=first_jenkins_at_build_of_current_pull_request,
    )

    try:
        first_jenkins_at_build_of_current_pull_request_id = first_jenkins_at_build_of_current_pull_request["data"]["build"]["number"]
    except KeyError as err:
        return create_error_singular_change_lead_time_result(f"Key {str(err)} cannot be found in the dict")
    except IndexError as err:
        return create_error_singular_change_lead_time_result(f"Unexpected data from {first_jenkins_at_build_of_current_pull_request_path} in {BITBUCKET_REPO_SLUG} acceptance job. Visit {first_jenkins_at_build_of_current_pull_request_path}")

    # seventh section
    # takes in first_jenkins_at_build_of_current_pull_request_id
    # outputs first_jenkins_pr_build_of_current_pull_request_start_timestamp & first_jenkins_pr_build_of_current_pull_request_duration_seconds

    first_jenkins_pr_build_of_current_pull_request_path = f"{JENKINS_PR_JOB_NAME}/api/xml?tree=builds[duration,timestamp,number,url,actions[causes[upstreamUrl,upstreamBuild]]]&xpath=/workflowJob/build/action/cause[upstreamBuild%20=%20%27{first_jenkins_at_build_of_current_pull_request_id}%27]/../.."

    logger.debug("making request to get the the id of the first prod build of the commit from the most recent PR", path=first_jenkins_pr_build_of_current_pull_request_path)
    first_jenkins_pr_build_of_current_pull_request = make_request(APIS.JENKINS, first_jenkins_pr_build_of_current_pull_request_path)

    if not first_jenkins_pr_build_of_current_pull_request["success"]:
        return handle_failed_network_request(APIS.JENKINS, first_jenkins_pr_build_of_current_pull_request_path, first_jenkins_pr_build_of_current_pull_request)

    logger.debug(
        "successful request to jenkins to",
        path=first_jenkins_pr_build_of_current_pull_request_path,
        response=first_jenkins_pr_build_of_current_pull_request,
    )

    try:
        first_jenkins_pr_build_of_current_pull_request_duration_seconds = int(first_jenkins_pr_build_of_current_pull_request["data"]["build"]["duration"])
        first_jenkins_pr_build_of_current_pull_request_start_timestamp = int(first_jenkins_pr_build_of_current_pull_request["data"]["build"]["timestamp"])
    except KeyError as err:
        return create_error_singular_change_lead_time_result(f"Key {str(err)} cannot be found in the dict")
    except IndexError as err:
        return create_error_singular_change_lead_time_result(f"Unexpected data from {first_jenkins_pr_build_of_current_pull_request_path} in {BITBUCKET_REPO_SLUG} production job. Visit {first_jenkins_pr_build_of_current_pull_request_path}")

    # eighth section
    
    first_jenkins_pr_build_of_current_pull_request_finish_timestamp = first_jenkins_pr_build_of_current_pull_request_start_timestamp + first_jenkins_pr_build_of_current_pull_request_duration_seconds

    first_jenkins_pr_build_of_current_pull_request_finish_datetime = jenkins_build_datetime({"timestamp": first_jenkins_pr_build_of_current_pull_request_finish_timestamp})

    logger.info(first_jenkins_pr_build_of_current_pull_request_finish_datetime - first_jenkins_build_of_current_pull_request_datetime)

    return SingularChangeLeadTimeResult(
        {
            "success": True,
            "data": {
                "startTime": first_jenkins_build_of_current_pull_request_datetime.isoformat(),
                "finishTime": first_jenkins_pr_build_of_current_pull_request_finish_datetime.isoformat()
            }
        }
    )


In [ ]:
def get_lead_time_for_changes():
    pull_requests_request_url = f"""/repositories/{BITBUCKET_WORKSPACE}/{BITBUCKET_REPO_SLUG}/pullrequests?state=MERGED&fields=values.id,values.title,values.state,values.merge_commit.hash,values.merge_commit.date,values.merge_commit.links.self.href,values.merge_commit.links.statuses.href,values.merge_commit.parents,values.merge_commit.parents.hash,values.merge_commit.parents.date,values.merge_commit.parents.links.self.href,values.merge_commit.parents.links.html.href,values.merge_commit.parents.links.statuses.href"""


    bitbucket_pull_requests_response = make_request(APIS.BITBUCKET, pull_requests_request_url)

    if not bitbucket_pull_requests_response["success"]:
        logger.info("bitbucket request errored out", url=pull_requests_request_url, response=bitbucket_pull_requests_response)
        return Response(
            status_code=status_codes.codes.SERVER_ERROR,
            content_type=content_types.APPLICATION_JSON,
            body={
                "message": bitbucket_pull_requests_response["message"]
                if "message" in bitbucket_pull_requests_response and bitbucket_pull_requests_response["message"]
                else "Error: a problem occured",
                "request_path": "/path",
            },
        )

    logger.debug("successfully got the pull requests response", response=bitbucket_pull_requests_response)

    try:
        most_recent_pull_request = bitbucket_pull_requests_response["data"]['values'][0]
    except KeyError as err:
        return Response(
            status_code=status_codes.codes.SERVER_ERROR,
            content_type=content_types.APPLICATION_JSON,
            body={
                "message": f"Key {str(err)} cannot be found in the dict",
            },
        )

    lead_time_changes = calculate_lead_time_for_changes(most_recent_pull_request)

    if not lead_time_changes["success"]:
        return Response(
            status_code=status_codes.codes.SERVER_ERROR,
            content_type=content_types.APPLICATION_JSON,
            body={
                "message": lead_time_changes["message"]
                if "message" in lead_time_changes and lead_time_changes["message"]
                else "Error: a problem occured",
                "request_path": "/path",
            },
        )

    return Response(
        status_code=status_codes.codes.OK,
        content_type=content_types.APPLICATION_JSON,
        body=lead_time_changes["data"]
    )

In [ ]:
finala = get_lead_time_for_changes()


print(f"\nHTTP Status: {finala.status_code}")
print(json.dumps(finala.body, indent=2))